# 总结：

## RESTORE本质和initializer()一样，两者可以无限执行和互相覆盖赋值结果。
## SAVE/RESTORE有严格的名字匹配机制，自动映射。
## 读取的时候，model中必须不少于当前定义的命名空间变量。
## 如果实在定义的变量多，tf.train.Saver()指定变量列表（实际情况：不可能不定义其他变量）

## placeholder不是variable，自然也不会被存,所以至少不会因为名字等原因干扰模型的读取。

## ema_restore = ema.variables_to_restore()，只能算指定了ema和普通变量的绑定关系，不是只有ema，其实包含了变量，并且不影响变量自身的读取

In [2]:
import tensorflow as tf
import numpy as np

# 简单例子，不过需要重启内核分别执行，因为scope会自动重命名，重命名以后就找不到参数了

In [3]:
with tf.variable_scope("variables_1"): 
	W = tf.Variable([[1,2,3],[3,4,5]], dtype = tf.float32, name='weights')
	b = tf.Variable([[1,2,3]],dtype = tf.float32,name='biases')
init = tf.global_variables_initializer()    
saver = tf.train.Saver()
with tf.Session() as sess:
	sess.run(init)
	save_path = saver.save(sess, "my_net/save_net_1.ckpt")
	print('Save to path:',save_path)
    

Save to path: my_net/save_net_1.ckpt


In [3]:
with tf.variable_scope("variables_1"):
	W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
    
init2 = tf.global_variables_initializer()

#no need init
loader = tf.train.Saver()
load_path = "my_net/save_net_1.ckpt"
with tf.Session() as sess:
	loader.restore(sess, load_path)
	print("W2:",sess.run(W2))
	print("b2:",sess.run(b2))

INFO:tensorflow:Restoring parameters from my_net/save_net_1.ckpt
W2: [[ 1.  2.  3.]
 [ 3.  4.  5.]]
b2: [[ 1.  2.  3.]]


# 因为命名空间冲突问题，需要重启内核
每一次都重启内核，重启内核，重启内核！！！！
因为此处没变量名，a会自动声明成Variable_1 Variable_2，save到的变量会悦来愈多，所以会产生很多误解。
有变量名，同样的名字还是如此，但是读取的时候那个没名字的变量也按Variable_1去找Variable_b，是找不到的。
save和restore是严格按变量名的，没有多余的适应功能，比如存一个，两个都用他来恢复，不存在的！！！！
Key Variable_1 not found in checkpoint！！！！！！！！！！！！！！！

总之，带名字，每次重启内核！！！！

In [1]:
#存
import tensorflow as tf
import numpy as np
import os
SAVE_PATH = './saver_test'#要用相对路径：Parent directory of saver_test doesn't exist, can't save.
a = tf.Variable(np.array([1,2,3]))
b = tf.Variable(np.array([3,1]))
saver = tf.train.Saver()
print(a)
print(b)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    if not os.path.exists(SAVE_PATH):
        os.mkdir(SAVE_PATH)
    saver.save(sess = sess, save_path = SAVE_PATH)
    print(sess.run(a))

<tf.Variable 'Variable:0' shape=(3,) dtype=int64_ref>
<tf.Variable 'Variable_1:0' shape=(2,) dtype=int64_ref>
[1 2 3]


In [2]:
#读取：初始化和恢复操作可以互相覆盖赋值操作
import tensorflow as tf
import numpy as np
import os
SAVE_PATH = './saver_test'
a = tf.Variable(np.array([2,3,4]))
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess = sess, save_path = SAVE_PATH)
    print(sess.run(a))
    tf.global_variables_initializer().run()
    print(sess.run(a))
    saver.restore(sess = sess, save_path = SAVE_PATH)
    print(sess.run(a))

INFO:tensorflow:Restoring parameters from ./saver_test
[1 2 3]
[2 3 4]
INFO:tensorflow:Restoring parameters from ./saver_test
[1 2 3]


In [1]:
import tensorflow as tf
import numpy as np
import os
SAVE_PATH = './saver_test'
#读取：对应原来的数据
#不指定名字很难对齐，python中的b可不是python中的那个b，现在无论a、b、c，都只找a。
#Variable_1和Variable_2吧
a = tf.Variable(np.array([2,3,4]))
# b = tf.Variable(np.array([1,1,1,1,1,1]))#形状不匹配 lhs shape= [6] rhs shape= [3]
c = tf.Variable(np.array([2,1]),name = 'Variable_1')#只存一个a，却能把a赋值给a和c？如果c形状不和a匹配会报错，但是ab都存了，这块会给c按b算，
# b = tf.Variable(np.array([2,1]))#

#这种默认名字，被前边b和c占了，最好单独起名，Key Variable_2_1 not found in checkpoint
#一旦给variable_b命名了，好像其他的也不能去读variable_1了，连锁反应
#可以看到，普通方法是无法用一个变量恢复到两个的，总之都有重名和重命名。
# d = tf.Variable(np.array([2]),name = 'Variable_1')
# print(b)
print(c)
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess = sess, save_path = SAVE_PATH)
    print(sess.run(a))

    saver.restore(sess = sess, save_path = SAVE_PATH)
    print(sess.run(a))
    print(sess.run(c))
#     print(sess.run(b))
#     print('d:',sess.run(d))

<tf.Variable 'Variable_1:0' shape=(2,) dtype=int64_ref>
INFO:tensorflow:Restoring parameters from ./saver_test
[1 2 3]
INFO:tensorflow:Restoring parameters from ./saver_test
[1 2 3]
[3 1]


NameError: name 'd' is not defined

# 带两个scope的例子：基本和前边相同，除了名字多个斜杠
(分别启动内核)
当然，你也可以自己加斜杠,仍然是被自动重命名
<tf.Variable 'variables_100/biases:0' shape=(1, 3) dtype=float32_ref>
<tf.Variable 'variables_101/biases:0' shape=(1, 3) dtype=float32_ref>
<tf.Variable 'variables_101/biases_1:0' shape=(1, 3) dtype=float32_ref>

#然后就是EMA

In [2]:
import tensorflow as tf
MOVING_AVERAGE_DECAY = 0.99
global_step = tf.Variable(0, trainable=False,name='global_step')
x = tf.placeholder(tf.float32, [1,2])#placeholder不是variable，自然也不会被存
with tf.variable_scope("variables_100"): 
	W = tf.Variable([[1,2,3],[3,4,5]], dtype = tf.float32, name='weights')
	b = tf.Variable([[1,2,3]],dtype = tf.float32,name='biases')
with tf.variable_scope("variables_101"):
	W = tf.Variable([[11,12,13],[13,14,15]], dtype = tf.float32, name='weights')
	b = tf.Variable([[11,12,13]],dtype = tf.float32,name='biases')
out_b = tf.Variable([[999,999,999]],dtype = tf.float32,name='variables_101/biases')

ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)#提前把EMA存起来
ema_op = ema.apply(tf.trainable_variables())


for ele2 in tf.all_variables():#比trainable多一个i哦global step：tf.trainable_variables():
	print(ele2.name)
# print('all variables:',get_vars)
saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    sess.run(ema_op)#应用一下
    save_path = saver.save(sess, "my_net/save_net_2.ckpt")
    print('Save to path:',save_path)

Instructions for updating:
Please use tf.global_variables instead.
variables_100/weights:0
variables_100/biases:0
variables_101/weights:0
variables_101/biases:0
variables_102/weights:0
variables_102/biases:0
global_step:0
variables_100_1/weights:0
variables_100_1/biases:0
variables_101_1/weights:0
variables_101_1/biases:0
variables_101/biases_1:0
variables_100/weights/ExponentialMovingAverage:0
variables_100/biases/ExponentialMovingAverage:0
variables_101/weights/ExponentialMovingAverage:0
variables_101/biases/ExponentialMovingAverage:0
variables_102/weights/ExponentialMovingAverage:0
variables_102/biases/ExponentialMovingAverage:0
variables_100_1/weights/ExponentialMovingAverage:0
variables_100_1/biases/ExponentialMovingAverage:0
variables_101_1/weights/ExponentialMovingAverage:0
variables_101_1/biases/ExponentialMovingAverage:0
variables_101/biases_1/ExponentialMovingAverage:0
Save to path: my_net/save_net_2.ckpt


{'variables_101/weights/ExponentialMovingAverage': <tf.Variable 'variables_101/weights:0' shape=(2, 3) dtype=float32_ref>, 
'variables_101/biases_1/ExponentialMovingAverage': <tf.Variable 'variables_101/biases_1:0' shape=(1, 3) dtype=float32_ref>, 
'variables_100/weights/ExponentialMovingAverage': <tf.Variable 'variables_100/weights:0' shape=(2, 3) dtype=float32_ref>, 
'variables_101/biases/ExponentialMovingAverage': <tf.Variable 'variables_101/biases:0' shape=(1, 3) dtype=float32_ref>, 
'variables_100/biases/ExponentialMovingAverage': <tf.Variable 'variables_100/biases:0' shape=(1, 3) dtype=float32_ref>, 
'Variable': <tf.Variable 'Variable:0' shape=() dtype=int32_ref>}



In [1]:
#with tf.variable_scope("variables_1"):
import tensorflow as tf
import numpy as np

with tf.variable_scope("variables_100"):
	W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
with tf.variable_scope("variables_101"):
	W22 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b22 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')

# out_b = tf.Variable([[66,666,66699]],dtype = tf.float32,name='variables_101/biases')
out_b = tf.Variable([[66,666,66699]],dtype = tf.float32,name='variables_101/biases_1')
init2 = tf.global_variables_initializer()


print(b2)
print(b22)
print(out_b)

#no need init
loader = tf.train.Saver()
load_path = "my_net/save_net_2.ckpt"
with tf.Session() as sess:
#3 way to initialize
#	sess.run(init)#error:not work,sequence problem
	#sess.run(init2)
#	sess.run(init2)
    loader.restore(sess, load_path)

    print("W2:",sess.run(W2))
    print("b2:",sess.run(b2))
    print("W22:",sess.run(W22))
    print("b22:",sess.run(b22))
#	print("biases2:",sess.run(b2))
    print(sess.run(out_b))



<tf.Variable 'variables_100/biases:0' shape=(1, 3) dtype=float32_ref>
<tf.Variable 'variables_101/biases:0' shape=(1, 3) dtype=float32_ref>
<tf.Variable 'variables_101/biases_1:0' shape=(1, 3) dtype=float32_ref>
INFO:tensorflow:Restoring parameters from my_net/save_net_2.ckpt
W2: [[ 1.  2.  3.]
 [ 3.  4.  5.]]
b2: [[ 1.  2.  3.]]
W22: [[ 11.  12.  13.]
 [ 13.  14.  15.]]
b22: [[ 11.  12.  13.]]
[[ 999.  999.  999.]]


# EMA读取
这个ema.variables_to_restore()其实是做映射取代原变量的，下边有例子。

In [1]:
#with tf.variable_scope("variables_1"):
import tensorflow as tf
import numpy as np

MOVING_AVERAGE_DECAY = 0.99
global_step = tf.Variable(0, trainable=False,name='global_step')

with tf.variable_scope("variables_100"):
	W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
with tf.variable_scope("variables_101"):
	W22 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b22 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')

# out_b = tf.Variable([[66,666,66699]],dtype = tf.float32,name='variables_101/biases')
out_b = tf.Variable([[66,666,66699]],dtype = tf.float32,name='variables_101/biases_1')

for ele2 in tf.global_variables():#比trainable多一个i哦global step：tf.trainable_variables():
	print(ele2.name)

ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
ema_restore = ema.variables_to_restore()
print('after ema.variables_to_restore()')
for ele2 in tf.all_variables():#比trainable多一个,global step                   tf.trainable_variables():
	print(ele2.name)
print(b2)
print(b22)
print(out_b)

print('ema_restore is ',ema_restore)
loader = tf.train.Saver(ema_restore)
load_path = "my_net/save_net_2.ckpt"
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    loader.restore(sess, load_path)

    print("W2:",sess.run(W2))
    print("b2:",sess.run(b2))
    print("W22:",sess.run(W22))
    print("b22:",sess.run(b22))
    #	print("biases2:",sess.run(b2))
    print(sess.run(out_b))
    
    print('after run restore')
#     for ele2 in tf.all_variables():#比trainable多一个i哦global step：tf.trainable_variables():
#         print(ele2.name)


global_step:0
variables_100/weights:0
variables_100/biases:0
variables_101/weights:0
variables_101/biases:0
variables_101/biases_1:0
after ema.variables_to_restore()
Instructions for updating:
Please use tf.global_variables instead.
global_step:0
variables_100/weights:0
variables_100/biases:0
variables_101/weights:0
variables_101/biases:0
variables_101/biases_1:0
<tf.Variable 'variables_100/biases:0' shape=(1, 3) dtype=float32_ref>
<tf.Variable 'variables_101/biases:0' shape=(1, 3) dtype=float32_ref>
<tf.Variable 'variables_101/biases_1:0' shape=(1, 3) dtype=float32_ref>
ema_restore is  {'variables_100/weights/ExponentialMovingAverage': <tf.Variable 'variables_100/weights:0' shape=(2, 3) dtype=float32_ref>, 'variables_101/biases_1/ExponentialMovingAverage': <tf.Variable 'variables_101/biases_1:0' shape=(1, 3) dtype=float32_ref>, 'variables_100/biases/ExponentialMovingAverage': <tf.Variable 'variables_100/biases:0' shape=(1, 3) dtype=float32_ref>, 'variables_101/biases/ExponentialMoving

{'variables_100/weights/ExponentialMovingAverage': <tf.Variable 'variables_100/weights:0' shape=(2, 3) dtype=float32_ref>, 
'variables_101/biases/ExponentialMovingAverage': <tf.Variable 'variables_101/biases:0' shape=(1, 3) dtype=float32_ref>, 
'variables_101/biases_1/ExponentialMovingAverage': <tf.Variable 'variables_101/biases_1:0' shape=(1, 3) dtype=float32_ref>, 
'variables_101/weights/ExponentialMovingAverage': <tf.Variable 'variables_101/weights:0' shape=(2, 3) dtype=float32_ref>, 
'variables_100/biases/ExponentialMovingAverage': <tf.Variable 'variables_100/biases:0' shape=(1, 3) dtype=float32_ref>, 'Variable': <tf.Variable 'Variable:0' shape=() dtype=int32_ref>}

# 有EMA的时候，恢复用谁的数值？取决于var_list

weights更新是不受ema影响的，训练的时候没什么影响，这是一定的。现在主要说预测的时候，怎么把ema调出来。

不指定var_list的时候，原来的'weights'仍然对应'weights'，如果指定了var_list，原来的'weights/ema'会替代'weights'，因为传进去的是个dict，手动的映射，所以说，其实模型改过命名，也不是一定就废弃了，还是可以手动映射来对上的，只不过比较繁琐，我这里就不弄了。

In [1]:
import tensorflow as tf
MOVING_AVERAGE_DECAY = 0.6#为了观察变化,数值越大，惯性越大，相比实际的W，增长越小越缓慢
#0.1的衰减
# [array([ 10.49953651,  20.49953651,  30.49953651], dtype=float32), 
#  array([ 10.38866425,  20.38866425,  30.38866425], dtype=float32)]
#0.6的衰减
#  [array([ 10.49953651,  20.49953651,  30.49953651], dtype=float32), 
#   array([ 10.34986782,  20.34986877,  30.34986877], dtype=float32)]


global_step = tf.Variable(0, trainable=False,name='global_step')
with tf.variable_scope("my_scope"): 
	W = tf.Variable([10,20,30], dtype = tf.float32, name='weights')
    
y = tf.constant([20,30,40],tf.float32)
loss = tf.reduce_mean(tf.square(W-y))
train_step = tf.train.AdamOptimizer(0.1).minimize(loss,global_step=global_step)
    
ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)#提前把EMA存起来
# ema_op = ema.apply(tf.trainable_variables())
ema_op = ema.apply([W])

with tf.control_dependencies([train_step, ema_op]):
    train_op = tf.no_op(name='train')

saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print('init,before assign:',sess.run(W))
    for i in range(5):
        sess.run(train_op)
        print('global_step is ',sess.run(global_step))
#         sess.run(train_step)
#         sess.run(ema_op)
        print('W and ema.average:',sess.run([W,ema.average(W)]))

    save_path = saver.save(sess, "my_net/save_net_3.ckpt")
    print('Save to path:',save_path)

init,before assign: [ 10.  20.  30.]
global_step is  1
W and ema.average: [array([ 10.10000038,  20.10000038,  30.10000038], dtype=float32), array([ 10.,  20.,  30.], dtype=float32)]
global_step is  2
W and ema.average: [array([ 10.19997215,  20.19997215,  30.19997215], dtype=float32), array([ 10.08181858,  20.08181763,  30.08181763], dtype=float32)]
global_step is  3
W and ema.average: [array([ 10.29989815,  20.29989815,  30.29989815], dtype=float32), array([ 10.170434  ,  20.17043304,  30.17043304], dtype=float32)]
global_step is  4
W and ema.average: [array([ 10.39975929,  20.39975929,  30.39975929], dtype=float32), array([ 10.26006317,  20.26006317,  30.26006317], dtype=float32)]
global_step is  5
W and ema.average: [array([ 10.49953651,  20.49953651,  30.49953651], dtype=float32), array([ 10.34986782,  20.34986877,  30.34986877], dtype=float32)]
Save to path: my_net/save_net_3.ckpt


In [1]:
#如果不指定var_list，普通的W还是按普通的W去提取。
#存[array([ 10.49953651,  20.49953651,  30.49953651], dtype=float32), array([ 10.34986782,  20.34986877,  30.34986877]
#取[ 10.49953651  20.49953651  30.49953651]
import tensorflow as tf
import numpy as np

MOVING_AVERAGE_DECAY = 0.6#0.99
global_step = tf.Variable(0, trainable=False,name='global_step')

with tf.variable_scope("my_scope"):
	W2 = tf.Variable([0,0,0], dtype=tf.float32,name='weights')

ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
ema_restore = ema.variables_to_restore()##w和w的ema，还有global，只能算指定了ema的绑定关系，不是只有ema，其实不影响变量自身的读取

# loader = tf.train.Saver(ema_restore)
loader = tf.train.Saver()
load_path = "my_net/save_net_3.ckpt"
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    loader.restore(sess, load_path)

    print("W2:",sess.run(W2))
#     print("W2:",sess.run([W2,ema.average(W2)]))
#     print('ema W is :',sess.run(ema_val))
    print(sess.run(ema_restore['my_scope/weights/ExponentialMovingAverage']))
    print('after run restore')
    print(tf.global_variables())



global_step:0
my_scope/weights:0
ema_restore: {'my_scope/weights/ExponentialMovingAverage': <tf.Variable 'my_scope/weights:0' shape=(3,) dtype=float32_ref>, 'global_step': <tf.Variable 'global_step:0' shape=() dtype=int32_ref>}
INFO:tensorflow:Restoring parameters from my_net/save_net_3.ckpt
W2: [ 10.49953651  20.49953651  30.49953651]
[ 10.49953651  20.49953651  30.49953651]
after run restore
[<tf.Variable 'global_step:0' shape=() dtype=int32_ref>, <tf.Variable 'my_scope/weights:0' shape=(3,) dtype=float32_ref>]


In [2]:
#如果指定var_list，普通的W按ema去提取。
#存[array([ 10.49953651,  20.49953651,  30.49953651], dtype=float32), array([ 10.34986782,  20.34986877,  30.34986877]
#取[ 10.34986782  20.34986877  30.34986877]
import tensorflow as tf
import numpy as np

MOVING_AVERAGE_DECAY = 0.6#0.99
global_step = tf.Variable(0, trainable=False,name='global_step')

with tf.variable_scope("my_scope"):
	W2 = tf.Variable([0,0,0], dtype=tf.float32,name='weights')

ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
ema_restore = ema.variables_to_restore()##w和w的ema，还有global，只能算指定了ema的绑定关系，不是只有ema，其实不影响变量自身的读取

print('ema_restore:',ema_restore)#这是一个映射，让EMA对应普通变量

loader = tf.train.Saver(ema_restore)
load_path = "my_net/save_net_3.ckpt"
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    loader.restore(sess, load_path)

    print("W2:",sess.run(W2))
#     print("W2:",sess.run([W2,ema.average(W2)]))
#     print('ema W is :',sess.run(ema_val))
    print(sess.run(ema_restore['my_scope/weights/ExponentialMovingAverage']))
    print('after run restore')
    print(tf.global_variables())



ema_restore: {'my_scope/weights/ExponentialMovingAverage': <tf.Variable 'my_scope/weights:0' shape=(3,) dtype=float32_ref>, 'my_scope_1/weights/ExponentialMovingAverage': <tf.Variable 'my_scope_1/weights:0' shape=(3,) dtype=float32_ref>, 'global_step_1': <tf.Variable 'global_step_1:0' shape=() dtype=int32_ref>, 'global_step': <tf.Variable 'global_step:0' shape=() dtype=int32_ref>}
INFO:tensorflow:Restoring parameters from my_net/save_net_3.ckpt


NotFoundError: Key my_scope_1/weights/ExponentialMovingAverage not found in checkpoint
	 [[Node: save_1/RestoreV2_3 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2_3/tensor_names, save_1/RestoreV2_3/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_3', defined at:
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance()

## 补一发映射的通用写法
{'var_name':tensor}
通过手动指定，颠倒两个变量的读取和赋值

tf.train.Saver的var_list其实支持两种写法，list和dict，dict中的key是文件中的变量名

In [1]:
import tensorflow as tf
import numpy as np

with tf.variable_scope("my_scope"):
	W2 = tf.Variable([1,2,3], dtype=tf.float32,name='weights')
	b2 = tf.Variable([2,3,4], dtype=tf.float32,name='biases')

saver = tf.train.Saver()
load_path = "my_net/save_net_4.ckpt"
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.save(sess, load_path)
    print("W2:",sess.run(W2))
    print("b2:",sess.run(b2))

W2: [ 1.  2.  3.]
b2: [ 2.  3.  4.]


In [1]:
import tensorflow as tf
import numpy as np

with tf.variable_scope("my_scope"):#颠倒一下
	b2 = tf.Variable([0,0,0], dtype=tf.float32,name='biases')#看name，这里本来也是b2和biases对应的
	W2 = tf.Variable([0,0,0], dtype=tf.float32,name='weightss')#不同名也无所谓
	W3 = tf.Variable([0,0,0], dtype=tf.float32,name='weightsss')

#另外，也不能够一个变量恢复到两个tensor，dict中key冲突覆盖了
restore_map = {'my_scope/weights':b2,'my_scope/biases':W2#,'my_scope/biases':W3
              }
loader = tf.train.Saver(restore_map)
load_path = "my_net/save_net_4.ckpt"
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    loader.restore(sess, load_path)
    print("W2:",sess.run(W2))
    print("b2:",sess.run(b2))
    print("W3:",sess.run(W3))

INFO:tensorflow:Restoring parameters from my_net/save_net_4.ckpt
W2: [ 2.  3.  4.]
b2: [ 1.  2.  3.]
W3: [ 0.  0.  0.]


# 如果定义很多参数，读取model时用不完，冗余，不影响。

In [2]:
import tensorflow as tf
import numpy as np
with tf.variable_scope("variables_100"):
	W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
loader = tf.train.Saver()
load_path = "my_net/save_net_2.ckpt"
with tf.Session() as sess:
	loader.restore(sess, load_path)
	print("W2:",sess.run(W2))
	print("b2:",sess.run(b2))

INFO:tensorflow:Restoring parameters from my_net/save_net_2.ckpt
W2: [[ 1.  2.  3.]
 [ 3.  4.  5.]]
b2: [[ 1.  2.  3.]]


# 如果原model定义参数少，读取model文件的参数多
就算variable有init操作也不允许。就是不行，not found 。因为你没有告诉Saver一个具体的var_list，Saver认为所有变量都需要去文件里找。

下边排列组合：怎样调整都不允许，包括把scope 102合并进101,或者改变量名。

In [1]:
import tensorflow as tf
import numpy as np
with tf.variable_scope("variables_100"):
	W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
with tf.variable_scope("variables_101"):
	W22 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b22 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
with tf.variable_scope("variables_102"):
	W23 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b23 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
# 	W23 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights23')
# 	b23 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases23')

init2 = tf.global_variables_initializer()

#no need init
loader = tf.train.Saver()
load_path = "my_net/save_net_2.ckpt"
with tf.Session() as sess:
	sess.run(init2)
	loader.restore(sess, load_path)
	print("W2:",sess.run(W2))
	print("b2:",sess.run(b2))
	print("W22:",sess.run(W22))
	print("b22:",sess.run(b22))



INFO:tensorflow:Restoring parameters from my_net/save_net_2.ckpt
W2: [[ 0.  1.  2.]
 [ 3.  4.  5.]]
b2: [[ 0.  1.  2.]]
W22: [[ 0.  1.  2.]
 [ 3.  4.  5.]]
b22: [[ 0.  1.  2.]]


# 如果tf.train.Saver指定var_list只取很少的参数呢？这就行的通了！
或者，allow_empty?不是一个东西

In [1]:

import tensorflow as tf
import numpy as np
with tf.variable_scope("variables_100"):
	W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
with tf.variable_scope("variables_101"):
	W22 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b22 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
with tf.variable_scope("variables_102"):
	W23 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
	b23 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
# 	W23 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights23')
# 	b23 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases23')

additional_variable = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='additional')

init2 = tf.global_variables_initializer()

#no need init
loader = tf.train.Saver([W2,b2,W22,b22])
# loader = tf.train.Saver(allow_empty = True)#这个不能解决，不是同一个问题。
load_path = "my_net/save_net_2.ckpt"
with tf.Session() as sess:

    sess.run(init2)
    loader.restore(sess, load_path)

    print("W2:",sess.run(W2))
    print("b2:",sess.run(b2))
    print("W22:",sess.run(W22))
    print("b22:",sess.run(b22))
    print('W23:',sess.run(W23))

INFO:tensorflow:Restoring parameters from my_net/save_net_2.ckpt


NotFoundError: Key additional not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/qw/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/qw/anaconda3/envs/py36/lib/p

In [ ]:
#same shape and same type and same name?
#W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights2')#wrong name
#b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases2')
#W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')
#b2 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases')
#b3 = tf.Variable(np.arange(3).reshape((1,3)), dtype=tf.float32,name='biases3')#wrong name too!


In [2]:
# #allow_empty是创建空graph用的，todo
# import tensorflow as tf
# import numpy as np
# # with tf.variable_scope("variables_100"):
# # 	W2 = tf.Variable(np.arange(6).reshape((2,3)), dtype=tf.float32,name='weights')

# saver = tf.train.Saver()
# save_path = "my_net/save_net_233.ckpt"
# with tf.Session() as sess:
# 	tf.global_variables_initializer().run()
# 	saver.save(sess, save_path)




In [3]:
help(loader.restore)

Help on method restore in module tensorflow.python.training.saver:

restore(sess, save_path) method of tensorflow.python.training.saver.Saver instance
    Restores previously saved variables.
    
    This method runs the ops added by the constructor for restoring variables.
    It requires a session in which the graph was launched.  The variables to
    restore do not have to have been initialized, as restoring is itself a way
    to initialize variables.
    
    The `save_path` argument is typically a value previously returned from a
    `save()` call, or a call to `latest_checkpoint()`.
    
    Args:
      sess: A `Session` to use to restore the parameters.
      save_path: Path where parameters were previously saved.



In [8]:
这烦人的重名问题怎么解决？get_variable？！！！！但是scope一样会自动重命名，不能解决，只能解决没scope的情况
tf.get_variable_scope就是另一种用途了。
help(tf.get_variable)#注意，第一个默认就是name，小心重名。关于data，应该用initializer吧。
help(tf.get_variable_scope)
# import tensorflow as tf
# with tf.variable_scope("variables_100"): 
# 	W = tf.get_variable(name='weights', dtype = tf.float32)
# 	b = tf.get_variable(name='biases',dtype = tf.float32)

# init = tf.global_variables_initializer()
# variable_set = tf.get_variable_scope()
# print(W)
# print(variable_set)
# saver = tf.train.Saver()
# with tf.Session() as sess:
#     sess.run(init)
#     save_path = saver.save(sess, "my_net/save_net_2.ckpt")
#     print('Save to path:',save_path)
#     print(sess.run(variable_set))